In [1]:
import sys
sys.path.append(r'C:\projects\DLCourse\DistanceGAN_boosting')

import pandas as pd
from models_repository import ModelsRepository
import dataset
from error_bound_calc_functions import samples_order_by_loss_from_filenames

In [2]:
# Load data
data_a, data_b = dataset.get_data('maps', test=False)
data_a_val, data_b_val = dataset.get_data('maps', test=True)

In [3]:
import os

def calc_losses(data_a, data_b, G1, G2):
    print(len(data_a))
    _, bounds, loss = samples_order_by_loss_from_filenames(data_a, data_b, G1[0], G2[0], False, 16)
    return bounds, loss

def get_version_bounds_losses(path, data_a, data_b):
    model_rep = ModelsRepository(models_path=path)
    G1, G2 = model_rep.get_models(), model_rep.get_models(False)
    bounds, loss = calc_losses(data_a, data_b, G1, G2)
    return bounds, loss
    
def get_bounds_losses(model, data_a, data_b, suffix=''):
    analysis_path = 'analysis\\{0}_{1}_maps.csv'.format(model, suffix)
    if os.path.exists(analysis_path):
        return pd.read_csv(analysis_path)
    
    base_path = r'models\{0}\maps'.format(model)
    paths = [os.path.join(base_path, o) for o in os.listdir(base_path) if os.path.isdir(os.path.join(base_path,o))]

    data = pd.DataFrame()
    for i, path in enumerate(paths):
        version = str(i + 1)
        if i==4:
            continue
        print('running bounds for {0} version {1}'.format(model, str(version)))
        bounds, loss = get_version_bounds_losses(path, data_a, data_b)
        data['bound' + version] = bounds
        data['loss' + version] = loss
    data.to_csv(analysis_path, index=False)
    return data

In [4]:
print('halving val')
stats_halving = get_bounds_losses('halving', data_a_val, data_b_val, 'val')
print('halving')
stats_halving_train = get_bounds_losses('halving', data_a, data_b, 'train')

# print('ada_boost val')
# stats_ada_boost = get_bounds_losses('ada_boost', data_a_val, data_b_val, 'val')
# print('halving_new_train val')
# stats_halving_new_train = get_bounds_losses('halving_new_train', data_a_val, data_b_val, 'val')
# print('ada_boost')
# stats_ada_boost_train = get_bounds_losses('ada_boost', data_a, data_b, 'train')
# print('halving_new_train')
# stats_halving_new_train_train = get_bounds_losses('halving_new_train', data_a, data_b, 'train')

halving val
halving


In [148]:
# read results
halving_val = pd.read_csv('analysis\\halving_val.csv').assign(method='halving', data='test')
ada_boost_val = pd.read_csv('analysis\\ada_boost_val.csv').assign(method='ada_boost', data='test')
halving_train = pd.read_csv('analysis\\halving_train.csv').assign(method='halving', data='train')
ada_boost_train = pd.read_csv('analysis\\ada_boost_train.csv').assign(method='ada_boost', data='train')
data = pd.concat([halving_val, halving_train, ada_boost_train, ada_boost_val])

In [17]:
%matplotlib inline

In [ ]:
halving_val = pd.read_csv('analysis\\halving_val.csv').assign(method='halving', data='test')
ada_boost_val = pd.read_csv('analysis\\ada_boost_val.csv').assign(method='ada_boost', data='test')
halving_train = pd.read_csv('analysis\\halving_train.csv').assign(method='halving', data='train')
ada_boost_train = pd.read_csv('analysis\\ada_boost_train.csv').assign(method='ada_boost', data='train')




In [5]:
stats_halving = stats_halving.assign(method='halving', data='test')
stats_halving_train = stats_halving_train.assign(method='halving', data='trains')
data = pd.concat([stats_halving, stats_halving_train])



In [28]:
0.14777/0.1640

0.9010365853658537

In [6]:
data.groupby(['data','method']).mean()[['loss1','loss2','loss3','loss4']].T.round(4)


data,test,trains
method,halving,halving
loss1,0.1249,0.1173
loss2,0.1334,0.1273
loss3,0.1254,0.1165
loss4,0.1281,0.1203


In [7]:
data.groupby(['data','method']).corr()

bound1    bound2    bound3    bound4     loss1  \
data   method                                                             
test   halving bound1  1.000000  0.272124 -0.268169 -0.178350  0.132200   
               bound2  0.272124  1.000000  0.407511  0.180291  0.286361   
               bound3 -0.268169  0.407511  1.000000  0.486321  0.490819   
               bound4 -0.178350  0.180291  0.486321  1.000000  0.291692   
               loss1   0.132200  0.286361  0.490819  0.291692  1.000000   
               loss2   0.008193  0.119054  0.456162  0.316674  0.928308   
               loss3   0.073410  0.218381  0.491354  0.309431  0.967446   
               loss4   0.096384  0.307475  0.466594  0.263170  0.933481   
trains halving bound1  1.000000  0.252102 -0.289952 -0.283053  0.113535   
               bound2  0.252102  1.000000  0.429525  0.363414  0.149230   
               bound3 -0.289952  0.429525  1.000000  0.601932  0.069365   
               bound4 -0.283053  0.363414  0.601932  1.000000  0.050210   
               loss1   0.113535  0.149230  0.069365  0.050210  1.000000   
               loss2   0.019935  0.070492  0.087280  0.056792  0.950488   
               loss3   0.046671  0.119851  0.098202  0.068375  0.971504   
               loss4   0.022213  0.070786  0.017083  0.019710  0.932676   

                          loss2     loss3     loss4  
data   method                                        
test   halving bound1  0.008193  0.073410  0.096384  
               bound2  0.119054  0.218381  0.307475  
               bound3  0.456162  0.491354  0.466594  
               bound4  0.316674  0.309431  0.263170  
               loss1   0.928308  0.967446  0.933481  
               loss2   1.000000  0.972559  0.844928  
               loss3   0.972559  1.000000  0.924423  
               loss4   0.844928  0.924423  1.000000  
trains halving bound1  0.019935  0.046671  0.022213  
               bound2  0.070492  0.119851  0.070786  
               bound3  0.087280  0.098202  0.017083  
               bound4  0.056792  0.068375  0.019710  
               loss1   0.950488  0.971504  0.932676  
               loss2   1.000000  0.971293  0.888650  
               loss3   0.971293  1.000000  0.941690  
               loss4   0.888650  0.941690  1.000000

In [27]:
data.groupby(['data','method']).corr().loc[['trains','test'],:,['bound1','bound2','bound3']][['loss1','loss2','loss3']].round(4)

loss1   loss2   loss3
data   method                                
test   halving bound1  0.8539  0.7900  0.6887
               bound2  0.7268  0.8474  0.6060
               bound3  0.6515  0.5488  0.8202
trains halving bound1  0.5467  0.4903  0.3840
               bound2  0.4480  0.6157  0.3109
               bound3  0.4493  0.3517  0.5513

In [115]:
data.groupby(['data','method']).mean().loc[['train','val']]#[['loss1','loss2','loss3']]

bound1    bound2    bound3    bound4    bound5     loss1  \
data  method                                                                    
train ada_boost    0.160949  0.103083  0.095555       NaN       NaN  0.150435   
      halving      0.160949  0.113241  0.124829  0.116475  0.119567  0.150435   
      halving_new  0.160949  0.173405  0.162180       NaN       NaN  0.150435   
val   ada_boost    0.165416  0.109263  0.101801       NaN       NaN  0.108206   
      halving      0.165416  0.118417  0.129041  0.122000  0.125198  0.108206   
      halving_new  0.165416  0.175483  0.164133       NaN       NaN  0.108206   

                      loss2     loss3     loss4     loss5  
data  method                                               
train ada_boost    0.145898  0.144403       NaN       NaN  
      halving      0.139361  0.137812  0.134443  0.126834  
      halving_new  0.145953  0.151256       NaN       NaN  
val   ada_boost    0.103897  0.099455       NaN       NaN  
      halving      0.100720  0.097868  0.097152  0.097841  
      halving_new  0.106841  0.106771       NaN       NaN

In [106]:
x.loc[['train','val'],:,['bound1','bound2','bound3','bound4','bound5']]

loss1     loss2     loss3     loss4     loss5
data  method                                                              
train ada_boost   bound1  0.617359  0.541425  0.552647       NaN       NaN
                  bound2  0.620049  0.564152  0.563086       NaN       NaN
                  bound3  0.615674  0.560039  0.588261       NaN       NaN
                  bound4       NaN       NaN       NaN       NaN       NaN
                  bound5       NaN       NaN       NaN       NaN       NaN
      halving     bound1  0.617359  0.481745  0.425383  0.376612  0.279940
                  bound2  0.574944  0.572345  0.390469  0.406871  0.235849
                  bound3  0.507415  0.407567  0.484129  0.352677  0.361107
                  bound4  0.471719  0.437589  0.360796  0.493281  0.243113
                  bound5  0.373511  0.283755  0.376837  0.266141  0.372648
      halving_new bound1  0.617359  0.492056  0.467667       NaN       NaN
                  bound2  0.500049  0.557529  0.383681       NaN       NaN
                  bound3  0.423545  0.321000  0.394227       NaN       NaN
                  bound4       NaN       NaN       NaN       NaN       NaN
                  bound5       NaN       NaN       NaN       NaN       NaN
val   ada_boost   bound1  0.455352  0.439652  0.433445       NaN       NaN
                  bound2  0.510050  0.501311  0.492387       NaN       NaN
                  bound3  0.495886  0.484222  0.483895       NaN       NaN
                  bound4       NaN       NaN       NaN       NaN       NaN
                  bound5       NaN       NaN       NaN       NaN       NaN
      halving     bound1  0.455352  0.402973  0.397903  0.376232  0.418805
                  bound2  0.511332  0.523923  0.461603  0.489604  0.499307
                  bound3  0.428943  0.393275  0.442044  0.368116  0.443674
                  bound4  0.461547  0.465258  0.430646  0.536105  0.474514
                  bound5  0.367611  0.329438  0.370251  0.299493  0.400478
      halving_new bound1  0.455352  0.394714  0.432769       NaN       NaN
                  bound2  0.475771  0.518051  0.472244       NaN       NaN
                  bound3  0.372282  0.315851  0.392469       NaN       NaN
                  bound4       NaN       NaN       NaN       NaN       NaN
                  bound5       NaN       NaN       NaN       NaN       NaN

In [65]:
x = data.groupby(['data','method']).corr()[['loss1','loss2','loss3','loss4','loss5']]
x.loc[('train'):('halving'),'bound1']



KeyError: 'the label [bound1] is not in the [columns]'

In [20]:
data.groupby(['data','method']).describe()

bound1        bound2        bound3  \
data  method                                                        
train ada_boost   count  24908.000000  24908.000000  24908.000000   
                  mean       0.160949      0.103083      0.095555   
                  std        0.022134      0.028765      0.027102   
                  min        0.106413      0.036373      0.037447   
                  25%        0.145389      0.082536      0.075651   
                  50%        0.159767      0.100761      0.092083   
                  75%        0.175226      0.120932      0.111546   
                  max        0.270153      0.246235      0.236008   
      halving     count  24908.000000  24908.000000  24908.000000   
                  mean       0.160949      0.113241      0.124829   
                  std        0.022134      0.029004      0.026261   
                  min        0.106413      0.056092      0.054515   
                  25%        0.145389      0.091256      0.105841   
                  50%        0.159767      0.107960      0.120923   
                  75%        0.175226      0.129923      0.139442   
                  max        0.270153      0.264279      0.280944   
      halving_new count  24908.000000  24908.000000  24908.000000   
                  mean       0.160949      0.173405      0.162180   
                  std        0.022134      0.024229      0.022785   
                  min        0.106413      0.118226      0.101034   
                  25%        0.145389      0.155395      0.145530   
                  50%        0.159767      0.170780      0.160711   
                  75%        0.175226      0.188069      0.176713   
                  max        0.270153      0.324346      0.310592   
val   ada_boost   count    200.000000    200.000000    200.000000   
                  mean       0.165416      0.109263      0.101801   
                  std        0.053585      0.061348      0.062039   
                  min        0.111628      0.044037      0.043504   
                  25%        0.144474      0.081580      0.075123   
                  50%        0.160880      0.104360      0.093779   
                  75%        0.176187      0.123561      0.117021   
                  max        0.841310      0.857201      0.864113   
      halving     count    200.000000    200.000000    200.000000   
                  mean       0.165416      0.118417      0.129041   
                  std        0.053585      0.063181      0.061556   
                  min        0.111628      0.059901      0.061272   
                  25%        0.144474      0.089806      0.102997   
                  50%        0.160880      0.106113      0.121494   
                  75%        0.176187      0.135280      0.140900   
                  max        0.841310      0.874610      0.893637   
      halving_new count    200.000000    200.000000    200.000000   
                  mean       0.165416      0.175483      0.164133   
                  std        0.053585      0.056850      0.054335   
                  min        0.111628      0.126098      0.104877   
                  25%        0.144474      0.149145      0.143978   
                  50%        0.160880      0.166966      0.159668   
                  75%        0.176187      0.190938      0.175970   
                  max        0.841310      0.857257      0.854184   

                                loss1         loss2         loss3  
data  method                                                       
train ada_boost   count  24908.000000  24908.000000  24908.000000  
                  mean       0.150435      0.145898      0.144403  
                  std        0.033720      0.032423      0.032761  
                  min        0.042785      0.042533      0.038375  
                  25%        0.126440      0.122733      0.121035  
                  50%        0.149536      0.144794      0.143325  
                  75%        0.17368

In [9]:
stats_ada_boost.mean()

bound1    0.165416
bound2    0.109263
bound3    0.101801
loss1     0.108206
loss2     0.103897
loss3     0.099455
dtype: float64

In [10]:
stats_halving_new_train.mean()

bound1    0.165416
bound2    0.175483
bound3    0.164133
loss1     0.108206
loss2     0.106841
loss3     0.106771
dtype: float64

In [12]:
data.corr()

,bound1,bound2,bound3,loss1,loss2,loss3,h_bound1,h_bound2,h_bound3,h_loss1,h_loss2,h_loss3
bound1,1.000000,0.942589,0.917261,0.617359,0.541425,0.552647,1.000000,0.835953,0.750859,0.617359,0.481745,0.425383
bound2,0.942589,1.000000,0.938870,0.620049,0.564152,0.563086,0.942589,0.872435,0.758763,0.620049,0.501178,0.435536
bound3,0.917261,0.938870,1.000000,0.615674,0.560039,0.588261,0.917261,0.919908,0.760392,0.615674,0.532639,0.420292
loss1,0.617359,0.620049,0.615674,1.000000,0.970139,0.969374,0.617359,0.574944,0.507415,1.000000,0.934546,0.910053
loss2,0.541425,0.564152,0.560039,0.970139,1.000000,0.978840,0.541425,0.534095,0.470885,0.970139,0.956203,0.938032
loss3,0.552647,0.563086,0.588261,0.969374,0.978840,1.000000,0.552647,0.558092,0.466539,0.969374,0.965544,0.930081
h_bound1,1.000000,0.942589,0.917261,0.617359,0.541425,0.552647,1.000000,0.835953,0.750859,0.617359,0.481745,0.425383
h_bound2,0.835953,0.872435,0.919908,0.574944,0.534095,0.558092,0.835953,1.000000,0.696032,0.574944,0.572345,0.390469
h_bound3,0.750859,0.758763,0.760392,0.507415,0.470885,0.466539,0.750859,0.696032,1.000000,0.507415,0.407567,0.484129
h_loss1,0.617359,0.620049,0.615674,1.000000,0.970139,0.969374,0.617359,0.574944,0.507415,1.000000,0.934546,0.910053


In [105]:
data['rank1'] = data.bound1.rank()
data['rank2'] = data.bound2.rank()
data['rank3'] = data.bound3.rank()

In [113]:
data.loc[0][data.loc[0].index.str.contains('bound')]

bound1               0.139532
bound2               0.085065
bound3               0.092330
lowest_bound_loss    0.092298
Name: 0, dtype: float64

In [158]:
data.bound1.describe()

count    24908.000000
mean         0.160949
std          0.022134
min          0.106413
25%          0.145389
50%          0.159767
75%          0.175226
max          0.270153
Name: bound1, dtype: float64

In [157]:
import numpy as np
np.exp(-data.bound1).describe()

count    24908.000000
mean         0.851543
std          0.018774
min          0.763263
25%          0.839268
50%          0.852342
75%          0.864686
max          0.899053
Name: bound1, dtype: float64

In [148]:
data.bound1.describe()

count    24908.000000
mean         0.160949
std          0.022134
min          0.106413
25%          0.145389
50%          0.159767
75%          0.175226
max          0.270153
Name: bound1, dtype: float64

In [147]:
data.bound1.std()

0.022134063338676825

In [145]:
data[['loss1','loss2','loss3']].min(axis=1).mean()

0.1421306270732698

In [25]:
data.groupby(data.bound1>data.bound1.median()).mean()

,bound1,bound2,bound3,loss1,loss2,loss3,h_bound1,h_bound2,h_bound3,h_loss1,h_loss2,h_loss3
bound1,,,,,,,,,,,,
False,0.143301,0.081527,0.075950,0.134089,0.132103,0.130181,0.143301,0.094284,0.109348,0.134089,0.126709,0.127050
True,0.178598,0.124639,0.115159,0.166781,0.159693,0.158626,0.178598,0.132197,0.140309,0.166781,0.152013,0.148574


In [27]:
data.mean().T

bound1      0.160949
bound2      0.103083
bound3      0.095555
loss1       0.150435
loss2       0.145898
loss3       0.144403
h_bound1    0.160949
h_bound2    0.113241
h_bound3    0.124829
h_loss1     0.150435
h_loss2     0.139361
h_loss3     0.137812
dtype: float64

In [24]:
data.groupby(data.loss1>data.loss1.median()).mean()

,bound1,bound2,bound3,loss1,loss2,loss3,h_bound1,h_bound2,h_bound3,h_loss1,h_loss2,h_loss3
loss1,,,,,,,,,,,,
False,0.150212,0.089056,0.082423,0.123198,0.120461,0.118734,0.150212,0.100118,0.114276,0.123198,0.113864,0.114050
True,0.171687,0.117110,0.108686,0.177672,0.171335,0.170072,0.171687,0.126364,0.135381,0.177672,0.164857,0.161574


In [ ]:
def get_loss(datum):
    index = datum[datum.index.str.startswith('loss')].min()
    return datum['loss' + index]

def get_lowest(datum):
    index = datum[datum.index.str.startswith('bound')].argmin()[-1]
    return datum['loss' + index]

def get_lowest_rank(datum):
    index = datum[datum.index.str.startswith('rank')].argmin()[-1]
    return datum['loss' + index]

In [114]:
data['lowest_bound_loss'] = data.apply(get_lowest, axis=1)

In [121]:
data['lowest_rank_loss'] = data.apply(get_lowest_rank, axis=1)

In [119]:
datum = data.loc[0]
index = datum[datum.index.str.startswith('rank')].argmin()[-1]
datum['loss' + index]

0.091554738581180573

In [137]:
data.groupby(data.bound2>data.bound2.median()).mean().T

bound2,False,True
bound1,0.144256,0.177643
bound2,0.080259,0.125907
bound3,0.092252,0.130644
loss1,0.133972,0.166898
loss2,0.131563,0.160233
loss3,0.129887,0.158920
lowest_bound_loss,0.131207,0.158787
rank1,6646.117191,18262.882809
rank2,6227.500000,18681.500000
rank3,6755.131725,18153.868275


In [141]:
data.sort_values('bound1', ascending=False)

,bound1,bound2,bound3,loss1,loss2,loss3,lowest_bound_loss,rank1,rank2,rank3,lowest_rank_loss
16336,0.270153,0.235734,0.239480,0.275224,0.269873,0.259321,0.269873,24908.0,24906.0,24908.0,0.269873
22609,0.260106,0.216256,0.204603,0.222521,0.207985,0.203517,0.203517,24907.0,24899.0,24862.0,0.203517
10199,0.258284,0.246235,0.232170,0.277935,0.263102,0.255705,0.255705,24906.0,24908.0,24907.0,0.277935
16129,0.251161,0.237442,0.228837,0.252939,0.233268,0.217354,0.217354,24905.0,24907.0,24906.0,0.252939
5987,0.248939,0.186516,0.181352,0.229066,0.215738,0.208769,0.208769,24904.0,24751.0,24600.0,0.208769
2255,0.248642,0.214600,0.217652,0.262204,0.259098,0.255412,0.259098,24903.0,24897.0,24897.0,0.259098
5977,0.248262,0.207192,0.209486,0.265810,0.249385,0.228487,0.249385,24902.0,24880.0,24880.0,0.249385
6287,0.247851,0.209671,0.206479,0.215890,0.198490,0.197536,0.197536,24901.0,24890.0,24868.0,0.197536
8772,0.245249,0.188932,0.201860,0.247591,0.192741,0.207449,0.192741,24900.0,24771.0,24844.0,0.192741
11464,0.244641,0.212637,0.217449,0.230566,0.230523,0.213913,0.230523,24899.0,24895.0,24896.0,0.230523
